# Carbon Emissions in Major US Cities
## by Adrian Dahlin

## Packages

In [2]:
import pandas as pd

# 1) Literature Review

A couple years ago I read the book "Green Metropolis" by David Owen. It lays out a pretty detailed case for why efficiences in buildings, transportation, etc make cities more sustainable than other forms of development. He argues that Manhattan is the most sustainable place in the world.

# 2) Gathering and Cleaning Data

## 2.A Urban population and population density from the U.S. Census for all cities/metro areas with populations greater than 100,000. Please note that the definition of a “city” is up to you: you may use MSAs, your own selected agglomerations of urban Counties, Primary MSAs, etc. You should justify your selection. You should use data from the 2000 decennial census to match the carbon data below.

Rationale for using MSAs: cities exist as part of a metropolitan region. To look at only the largest municipalities would be to discount the effect cities have on their suburbs. The structure of a city–including its transportation system, land use choices, geography, economy, and policies–influences the development of suburbs and therefore, the carbon footprint of those suburbs. A city could have low carbon emissions but if a high number of workers commute in by car from the suburbs, the region overall might have a higher climate impact.

First, getting the MSA data with population numbers.

In [98]:
!curl -O "https://www.census.gov/population/www/cen2000/briefs/phc-t29/tables/tab03a.csv"
# did some munging in Excel and renamed to "MSA data.csv"
census1 = pd.read_csv("MSA data.csv")
print(len(census1))
census1.head(3)

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 90003  100 90003    0     0   414k      0 --:--:-- --:--:-- --:--:--  455k
974


,Metro/ Micro Area Code,Metro Div. Code,2000 Pop. Rank,Metropolitan Statistical Area Metropolitan Division Micropolitan Statistical Area,Legal/Statistical Area Description,2000 Population,1990 Population,# Change 1990 to 2000 Percent,% Change 1990 to 2000 Percent
0,35620,NaN,1,"New York-Northern New Jersey-Long Island, NY-N...",Metropolitan Statistical Area,"18,323,002","16,846,046","1,476,956",8.8
1,35620,20764.0,(x),".Edison, NJ",Metropolitan Division,"2,173,869","1,898,386","275,483",14.5
2,35620,35004.0,(x),".Nassau-Suffolk, NY3/",Metropolitan Division,"2,753,913","2,609,212","144,701",5.5


In [10]:
census1.columns

Index([u'Metro/\rMicro Area\rCode', u'Metro\rDiv.\rCode', u'2000\rPop. \rRank',
       u'Metropolitan Statistical Area\rMetropolitan Division\rMicropolitan Statistical Area',
       u'Legal/Statistical\rArea Description', u'2000 Population',
       u'1990 Population', u'# Change 1990 to 2000 Percent',
       u'% Change 1990 to 2000 Percent'],
      dtype='object')

Selecting only MSAs of at least population 100,000

In [24]:
census2 = census1[census1["Legal/Statistical\rArea Description"] == "Metropolitan Statistical Area"]
print(len(census2))
census2.head(3)

370


,Metro/ Micro Area Code,Metro Div. Code,2000 Pop. Rank,Metropolitan Statistical Area Metropolitan Division Micropolitan Statistical Area,Legal/Statistical Area Description,2000 Population,1990 Population,# Change 1990 to 2000 Percent,% Change 1990 to 2000 Percent
0,35620,NaN,1,"New York-Northern New Jersey-Long Island, NY-N...",Metropolitan Statistical Area,"18,323,002","16,846,046","1,476,956",8.8
5,31100,NaN,2,"Los Angeles-Long Beach-Santa Ana, CA",Metropolitan Statistical Area,"12,365,627","11,273,720","1,091,907",9.7
8,16980,NaN,3,"Chicago-Naperville-Joliet, IL-IN-WI",Metropolitan Statistical Area,"9,098,316","8,182,076","916,240",11.2


In [25]:
census2['2000 Population'].dtype

dtype('O')

In [42]:
census2['2000 Population'].replace(regex=True, inplace=True, to_replace=r',', value=r'')
census2['2000 Population'] = census2['2000 Population'].astype(int)

census3 = census2[census2["2000 Population"] >= 100000]
print(len(census3))
# and out of curiosity, counting how many MSAs have over 1,000,000 people
censusMil = census2[census2["2000 Population"] >= 1000000]
print(len(censusMil))
census3.head(3)

340
50


/Users/adriandahlin/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


,Metro/ Micro Area Code,Metro Div. Code,2000 Pop. Rank,Metropolitan Statistical Area Metropolitan Division Micropolitan Statistical Area,Legal/Statistical Area Description,2000 Population,1990 Population,# Change 1990 to 2000 Percent,% Change 1990 to 2000 Percent
0,35620,NaN,1,"New York-Northern New Jersey-Long Island, NY-N...",Metropolitan Statistical Area,18323002,"16,846,046","1,476,956",8.8
5,31100,NaN,2,"Los Angeles-Long Beach-Santa Ana, CA",Metropolitan Statistical Area,12365627,"11,273,720","1,091,907",9.7
8,16980,NaN,3,"Chicago-Naperville-Joliet, IL-IN-WI",Metropolitan Statistical Area,9098316,"8,182,076","916,240",11.2


Now getting the MSA data including all the counties encompassed in each.

In [53]:
# download from https://www.dol.gov/owcp/regs/feeschedule/fee/fs04ctst.xls
# brief munging in Excel
MSAcounties = pd.read_csv('MSAs with counties.csv')
print(len(MSAcounties))
MSAcounties.head()

1885


,﻿CBSA Code,Metro Division Code,CSA Code,CBSA Title,Metropolitan/Micropolitan Statistical Area,Metropolitan Division Title,CSA Title,County/County Equivalent,State Name,FIPS State Code,FIPS County Code,Central/Outlying County
0,10100,NaN,NaN,"Aberdeen, SD",Micropolitan Statistical Area,NaN,NaN,Brown County,South Dakota,46.0,13.0,Central
1,10100,NaN,NaN,"Aberdeen, SD",Micropolitan Statistical Area,NaN,NaN,Edmunds County,South Dakota,46.0,45.0,Outlying
2,10140,NaN,NaN,"Aberdeen, WA",Micropolitan Statistical Area,NaN,NaN,Grays Harbor County,Washington,53.0,27.0,Central
3,10180,NaN,NaN,"Abilene, TX",Metropolitan Statistical Area,NaN,NaN,Callahan County,Texas,48.0,59.0,Outlying
4,10180,NaN,NaN,"Abilene, TX",Metropolitan Statistical Area,NaN,NaN,Jones County,Texas,48.0,253.0,Outlying


In [104]:
MSA1 = MSAcounties[MSAcounties['Metropolitan/Micropolitan Statistical Area'] == "Metropolitan Statistical Area"]
MSA1.rename(columns={'County/County Equivalent': 'County'}, inplace=True)
print(MSA1.columns)
MSA1.head(3)

Index([u'﻿CBSA Code', u'Metro Division Code', u'CSA Code', u'CBSA Title',
       u'Metropolitan/Micropolitan Statistical Area',
       u'Metropolitan Division Title', u'CSA Title', u'County', u'State Name',
       u'FIPS State Code', u'FIPS County Code', u'Central/Outlying County'],
      dtype='object')


,﻿CBSA Code,Metro Division Code,CSA Code,CBSA Title,Metropolitan/Micropolitan Statistical Area,Metropolitan Division Title,CSA Title,County,State Name,FIPS State Code,FIPS County Code,Central/Outlying County
3,10180,NaN,NaN,"Abilene, TX",Metropolitan Statistical Area,NaN,NaN,Callahan County,Texas,48.0,59.0,Outlying
4,10180,NaN,NaN,"Abilene, TX",Metropolitan Statistical Area,NaN,NaN,Jones County,Texas,48.0,253.0,Outlying
5,10180,NaN,NaN,"Abilene, TX",Metropolitan Statistical Area,NaN,NaN,Taylor County,Texas,48.0,441.0,Central


In [105]:
MSA1.drop(['Metro Division Code', 'CSA Code', 'Metropolitan Division Title',
           'CSA Title', 'County', 'Central/Outlying County', 'FIPS State Code',
           'Metropolitan/Micropolitan Statistical Area'], axis=1, inplace=True)
print(MSA1['FIPS County Code'].dtype)
MSA1.head(3)

float64


/Users/adriandahlin/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


,﻿CBSA Code,CBSA Title,State Name,FIPS County Code
3,10180,"Abilene, TX",Texas,59.0
4,10180,"Abilene, TX",Texas,253.0
5,10180,"Abilene, TX",Texas,441.0


## 2.B Per capita carbon emissions from the ASU Vulcan Project (http://vulcan.project.asu.edu/). This dataset provides County-level carbon emissions from multiple sectors. You should download annual data for 2002 and “total” from the “sectors” option. You will need to normalize total emissions by County population. You will also need to aggregate multiple Counties to geographically align to your Census data for each city.

Potential problem:
The two sheets in the file show different quantities in the Total columns. Those in the "per capita" sheet are about an order of magnitude higher.

In [61]:
# Used Excel to combine data from different sheets.
# Copied Pop 2000 from the "per capita" sheet and pasted it onto the "county_sector" sheet,
# which had the FIPS code we need for merging.

carbon = pd.read_csv('CarbonPerCapWithFIPS.csv')
carbon.head()

,﻿State,County,FIPS,Total,Unnamed: 4,Commercial,Industrial,Residential,Electricity Prod,Onroad,Cement,Aircraft,Airborne,Nonroad,Pop 2000
0,AL,Autauga,1001.0,0.256167,NaN,0.003001,0.004434,0.010148,0.155508,0.074761,0.0,0.000976,0.002453,0.004886,43671.0
1,AL,Baldwin,1003.0,0.450031,NaN,0.015348,0.011884,0.031948,0.000000,0.278709,0.0,0.017842,0.044849,0.049451,140415.0
2,AL,Barbour,1005.0,0.099370,NaN,0.001905,0.008555,0.006284,0.000000,0.057143,0.0,0.005469,0.013747,0.006268,29038.0
3,AL,Bibb,1007.0,0.045945,NaN,0.000769,0.001622,0.004614,0.000000,0.035746,0.0,0.000380,0.000956,0.001858,20826.0
4,AL,Blount,1009.0,0.114776,NaN,0.001951,0.009059,0.011162,0.000000,0.087401,0.0,0.000298,0.000749,0.004155,51024.0


In [67]:
carbon.drop(['Unnamed: 4', 'Commercial', 'Industrial',
                   'Residential', 'Electricity Prod', 'Onroad',
                   'Cement', 'Aircraft', 'Nonroad', 'Airborne'], axis=1, inplace=True)
carbon.columns

Index([u'﻿State', u' County', u' FIPS', u' Total', u'Pop 2000'], dtype='object')

In [82]:
carbon['PerCap'] = carbon[' Total'] / carbonPerCap['Pop 2000']
carbon.rename(columns={' FIPS': 'FIPS County Code'}, inplace=True)
#carbon['FIPS County Code'].dropna(inplace=False)
print(len(carbon))
carbon.head(3)

3142


,﻿State,County,FIPS County Code,Total,Pop 2000,PerCap
0,AL,Autauga,1001.0,0.256167,43671.0,0.000006
1,AL,Baldwin,1003.0,0.450031,140415.0,0.000003
2,AL,Barbour,1005.0,0.099370,29038.0,0.000003


In [83]:
carbon['FIPS County Code'].dtype

dtype('float64')

## 2.C Merges

My plan is to merge the carbon data with the second census dataset - the one that has the individual counties. I wrangled the carbon emissions data so that it included FIPS code, which the census data also has, enabling a merge. That merge is not working, however, and I'm stumped after a little troubleshooting. The FIPS code given in the census data is 2-3 digits, while the FIPS codes in the carbon data are four digits (at a quick glance anyway).

The next step would be to use something like groupby to add up the population and carbon emissions for all counties in each MSA. Then simple division shows carbon per capita in each MSA.

In [112]:
MSA1['FIPS County Code'].head()

3      59.0
4     253.0
5     441.0
9       3.0
10      5.0
Name: FIPS County Code, dtype: float64

In [111]:
carbon['FIPS County Code'].head()

0    1001.0
1    1003.0
2    1005.0
3    1007.0
4    1009.0
Name: FIPS County Code, dtype: float64

In [92]:
merge1 = pd.merge(MSA1, carbon, on=['FIPS County Code'])
merge1.head()

,﻿CBSA Code,CBSA Title,Metropolitan/Micropolitan Statistical Area,State Name,FIPS State Code,FIPS County Code,﻿State,County,Total,Pop 2000,PerCap


Also need to merge existing data with the following, to get population density per MSA:

In [107]:
densityData = pd.read_csv('MSAWithLandAreaAndPop.csv')
densityData.head(3)

,GEO.id,GEO.id2,GEO.display-label,GCT_STUB.target-geo-id,GCT_STUB.target-geo-id2,GCT_STUB.display-label,GCT_STUB.display-label.1,HC01,HC02,HC04,HC05,HC06,HC08,HC09
0,Id,Id2,Geography,Target Geo Id,Target Geo Id2,Geographical Area,Geographical Area,Population,Housing units,Area in square miles - Total area,Area in square miles - Water area,Area in square miles - Land area,Density per square mile of land area - Population,Density per square mile of land area - Housing...
1,0100000US,NaN,United States,0100000US,NaN,United States,United States,281421906,115904641,3794083.06,256644.62,3537438.44,79.6,32.8
2,0100000US,NaN,United States,0100052US,NaN,United States - In metropolitan area,In metropolitan area,225981679,90812960,764595.13,58805.54,705789.59,320.2,128.7


# 3) Analysis

I'm running out of time so I'm showing methodology here. These are the methods I'd use if I could get that merge to work.

## 3.A Scatterplots

In [ ]:
emissions = merge1['column name']
density = merge1['other column name']

In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(?, ?)
ax.plot(density, emissions, '.', label='=  Metropolitan Statistical Area')
ax.set_title('Relationship Between Population Density of a Metropolitan Area\
             and Its Carbon Emissions Per Capita')
ax.set_ylabel('Carbon Dioxide Emissions Per Capita', fontsize=18)
ax.set_xlabel('Population Density (unit)', fontsize=18)
plt.legend(loc=?)
plt.show()

## 3.B  Ranking Cities by Per Capita Emissions

Simple sorting of a dataframe.

## 3.C Regression 

I'd probably try linear regression on the simple scatterplot first, then do a log of the x axis since the population density varies so much and has so many values clustered close to the origin.

## 3.D Other charts/graphs/maps

I'd like to create a map in Carto or ArcMap once the single merged table is available. Make it a map with two features: a choropleth of population density plus a proportionately-sized dot visualization on top of the MSAs that represents emissions per capita.

# 4) Discussion

I expect that the analysis would support the conclusions reached by David Owen: higher density means lower emissions per person. As far as the question of "optimal city size" I think you need to look at metros outside the US to analyze this. We have only one mega-city (NY), but there are several others of about the same size and one or two significantly larger. With that amount of data you MIGHT be able to make conclusions about optimal size - unless Tokyo and Jakarta have lower emissions per capita than the rest.

It also would be interesting to do a non-linear analysis (visual assessment of the scatterplot might suggest whether or not this would be worthwhile). This might produce recommendations for comparative carbon benefits of various city densities along the curve.